<a href="https://colab.research.google.com/github/umedadada/nikkei_software_202407/blob/master/2024_nikkei_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ライブラリの読み込み
import pandas as pd #データを取り扱うためのライブラリ
import plotly.express as px #データを可視化するためのライブラリ

In [ ]:
# データの読み込み
raw_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FEH_00500215_240423175115.csv',encoding="shift-jis")
# データのカラム確認
raw_df.info()
# データの表示
raw_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95040 entries, 0 to 95039
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   cat01_code              95040 non-null  int64 
 1   結果樹面積・10a当たり収量・収穫量・出荷量  95040 non-null  object
 2   cat02_code              95040 non-null  int64 
 3   果樹調査品目                  95040 non-null  object
 4   area_code               95040 non-null  int64 
 5   全国、都府県、全国農業地域           95040 non-null  object
 6   time_code               95040 non-null  int64 
 7   時間軸（年次）                 95040 non-null  object
 8   unit                    95040 non-null  object
 9   value                   95040 non-null  object
dtypes: int64(4), object(6)
memory usage: 7.3+ MB


,cat01_code,結果樹面積・10a当たり収量・収穫量・出荷量,cat02_code,果樹調査品目,area_code,全国、都府県、全国農業地域,time_code,時間軸（年次）,unit,value
0,100,結果樹面積,300,りんご,0,全国,2016000000,2016年,ha,36800
1,100,結果樹面積,300,りんご,0,全国,2015000000,2015年,ha,37000
2,100,結果樹面積,300,りんご,0,全国,2014000000,2014年,ha,37100
3,100,結果樹面積,300,りんご,0,全国,2013000000,2013年,ha,37200
4,100,結果樹面積,300,りんご,0,全国,2012000000,2012年,ha,37400
...,...,...,...,...,...,...,...,...,...,...
95035,170,出荷量,440,りんご_その他,47000,沖縄県,1977000000,1977年,t,…
95036,170,出荷量,440,りんご_その他,47000,沖縄県,1976000000,1976年,t,…
95037,170,出荷量,440,りんご_その他,47000,沖縄県,1975000000,1975年,t,…
95038,170,出荷量,440,りんご_その他,47000,沖縄県,1974000000,1974年,t,…


In [ ]:
# データの前処理
df = raw_df[(raw_df['結果樹面積・10a当たり収量・収穫量・出荷量']=='収穫量') & #指標を「収穫量」に絞る
            (raw_df['果樹調査品目']=='りんご')] #品目を「りんご」に絞る
df = df[['時間軸（年次）','結果樹面積・10a当たり収量・収穫量・出荷量','果樹調査品目','全国、都府県、全国農業地域','value']] #カラムを絞る
df.columns = ['year','val_type','type_fruit','area','value'] #カラム名を変更する
df = df.assign(value=df['value'].replace('−',0))# 数値以外の値を除去
df = df.assign(value=df['value'].replace('…',0))# 数値以外の値を除去
df = df.assign(value=df['value'].replace('X',0))# 数値以外の値を除去
df = df.assign(value = df['value'].astype('int64')) #数値カラムの型変換をする
df = df.sort_values(by=['year','value'],ascending=True) #西暦でソートする
df

,year,val_type,type_fruit,area,value
32295,1973年,収穫量,りんご,東京都,0
32515,1973年,収穫量,りんご,福井県,0
32911,1973年,収穫量,りんご,大阪府,0
32955,1973年,収穫量,りんご,兵庫県,0
33043,1973年,収穫量,りんご,和歌山県,0
...,...,...,...,...,...
31812,2016年,収穫量,りんご,岩手県,43800
31944,2016年,収穫量,りんご,山形県,45700
32560,2016年,収穫量,りんご,長野県,142100
31768,2016年,収穫量,りんご,青森県,447800


In [ ]:
# 調査年の範囲を調べる
print([min(df['year']),max(df['year'])])

['1973年', '2016年']


In [ ]:
# 可視化のためのデータ加工
## 条件の絞り込み
viz_df = df[(df['area']!='全国')& #47都道府県のデータだけに絞る
            (df['year']=='2016年')] #2016年のデータに絞る

## 軸の名前を明示的に指定して可視化
fig = px.pie(viz_df, values='value', names='area',color='area',title='図. 2016年の国内りんご収穫量の内訳')
fig.update_traces(textinfo='label+percent',textposition='inside')

fig

In [ ]:
# 可視化のためのデータ加工
top5_prefecture = viz_df.nlargest(5,'value')['area'] #収穫量の多い都道府県TOP5を特定する
viz_df.loc[~viz_df['area'].isin(top5_prefecture), 'area'] = 'その他' #TOP5以外の都道府県を「その他」に変更する

## 総収穫量を算出
sum_val=sum(viz_df['value'])
## コンマ表示に整形する
comma_sum_val = "{:,}".format(sum_val)

## 可視化
fig = px.pie(viz_df
             , values='value'
             , names='area',color='area'
               ,title='図. 2016年の国内りんご収穫量の内訳（総収穫量：{}t ）'.format(comma_sum_val))
fig.update_traces(textinfo='label+percent',textposition='inside')
fig

In [ ]:
# ## 総収穫量を算出
# sum_val=sum(viz_df['value'])
# ## コンマ表示に整形する
# comma_sum_val = "{:,}".format(sum_val)
fig = px.pie(viz_df, values='value', names='area',color='area'
             ,title='図. 2016年の国内りんご収穫量の内訳（総収穫量：{}t ）'.format(comma_sum_val)
             ,color_discrete_map={
              '青森県':'red'
              ,'長野県':'pink'
              ,'山形県':'gray'
              ,'岩手県':'gray'
              ,'福島県':'gray'
              ,'その他':'gray'
              }
             )
fig.update_traces(textinfo='label+percent',textposition='inside')

In [ ]:
# ## 条件の絞り込み
# viz_df = df[(df['area']=='全国')]

# px.line(viz_df,x='year',y='value')

In [ ]:
# 可視化のためのデータ加工
## 条件の絞り込み
viz_df = df[(df['area']!='全国')&
            (df['year']=='2016年')]

## 総収穫量を算出
sum_val=sum(viz_df['value'])
## コンマ表示に整形する
comma_sum_val = "{:,}".format(sum_val)
# ツリーマップの作成
fig = px.treemap(viz_df, path=['area'], values='value',title='図. 2016年の国内りんご収穫量の内訳（総収穫量：{}t ）'.format(comma_sum_val))
fig.update_traces(textinfo='label+percent root+value')
fig

In [ ]:
# 可視化のためのデータ加工
## 条件の絞り込み
viz_df = df[(df['area']!='全国')&
            (df['year'].isin(['2016年','1973年']))]

# ツリーマップの作成
fig = px.treemap(viz_df, path=['year','area'], values='value',title='図. 1973年と2016年の国内りんご収穫量の内訳の比較')
fig.update_traces(textinfo='label+percent root+value')
fig

In [ ]:
#絞り込み
viz_df = df[(df['area']!='全国')] #条件の絞り込み

# 各列の合計を計算
col_sum = viz_df.groupby('year')['value'].sum()

# 各列の合計で各要素を割る
normalized_df = viz_df.copy()  # データを変更せずにコピーする
for year, sum_value in col_sum.items():
    normalized_df.loc[normalized_df['year'] == year, 'value'] /= sum_value

# valueの値に100をかける
normalized_df['value'] *= 100

fig=px.bar(normalized_df, x='year', y='value',color='area')
fig.update_layout(
    title='図. 各都道府県のりんご収穫量の割合の推移',
    xaxis=dict(title='調査年'),
    yaxis=dict(title='パーセント'))
fig

In [ ]:
# viz_df = df[(df['area']=='全国')] #条件の絞り込み
# fig=px.bar(viz_df, x='year', y='value',color='area') #折れ線を描画
# fig.update_layout(
#     title='図. 全国のりんご収穫量の推移',
#     xaxis=dict(title='調査年'),
#     yaxis=dict(title='りんごの収穫量'))
# fig

In [ ]:
# データの読み込み
raw_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ctr_data.csv')
# データのカラム確認
raw_df.info()
# データの表示
raw_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70010 entries, 0 to 70009
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       70010 non-null  object
 1   user_id    70010 non-null  object
 2   flg_click  70010 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


,date,user_id,flg_click
0,2024-04-29,Bp5vKRUi,0
1,2024-04-29,eLpdHMvz,0
2,2024-04-29,fonZn939,0
3,2024-04-29,dduNt3cJ,0
4,2024-04-29,EC5eb9UN,0
...,...,...,...
70005,2024-05-06,rQy48904,1
70006,2024-05-06,GIxFSdTy,0
70007,2024-05-06,kHJmZ11x,1
70008,2024-05-06,UjbRrFUl,0


In [ ]:
# 可視化のためのデータ加工
df=raw_df.groupby(['date','flg_click']).nunique() #配信日とクリックフラグごとにデータをカウントする
df=df.reset_index(['date','flg_click']) #indexを解除する
df['flg_click'] = df['flg_click'].map({0: 'noneclick', 1: 'click'}) # flg_clickに基づいて文字列を代入
df=df.pivot_table(index='date',columns='flg_click', values='user_id').reset_index() #縦持ちデータを横持ちに変換する
df['sum']=df['click']+df['noneclick'] #日毎のメルマガの配信総数を計算する
df['ctr']=df['click']/df['sum'] #CTRを計算する
df

flg_click,date,click,noneclick,sum,ctr
0,2024-04-29,1113,8887,10000,0.1113
1,2024-04-30,1110,8890,10000,0.1110
2,2024-05-01,1089,8911,10000,0.1089
3,2024-05-02,1139,8861,10000,0.1139
4,2024-05-03,1122,8878,10000,0.1122
5,2024-05-04,1119,8881,10000,0.1119
6,2024-05-05,1076,8924,10000,0.1076
7,2024-05-06,2,8,10,0.2000


In [ ]:
#CTRを可視化
fig=px.line(df,x='date',y='ctr')
fig.update_layout(
    title='図. GW期間中のCTRの推移',
    xaxis=dict(title='メルマガ配信日'),
    yaxis=dict(title='CTR'))

In [ ]:
#メルマガの総配信数を可視化
fig=px.line(df,x='date',y='sum')
fig.update_layout(
    title='図. GW期間中のメルマガの配信総数の推移',
    xaxis=dict(title='メルマガ配信日'),
    yaxis=dict(title='メルマガの配信総数'))